In [2]:
import os
import dspy
from dsp.utils import deduplicate
from dotenv import load_dotenv
load_dotenv()

turbo = dspy.OpenAI(model="gpt-3.5-turbo", api_key=os.getenv("OPENAI_API_KEY"))
dspy.settings.configure(lm=turbo)

d:\ThirdSummer\Something\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from dspy import ColBERTv2

colber_wiki = ColBERTv2(url="http://20.102.90.50:2017/wiki17_abstracts")
dspy.settings.configure(lm=turbo, rm=colber_wiki)

In [16]:
question = "Acid Mine Drainage"
retriever = dspy.Retrieve(k=5)
raw = retriever(question)

for passage in raw.passages:
    print(passage)


rqa = dspy.Predict("question, context -> answer")
response = rqa(question=question, context=raw.passages)

Acid mine drainage | Acid mine drainage, acid and metalliferous drainage (AMD), or acid rock drainage (ARD) is the outflow of acidic water from metal mines or coal mines.
Acidophiles in acid mine drainage | The outflow of acidic liquids and other pollutants from mines is often catalysed by acid-loving microorganisms; these are the acidophiles in acid mine drainage.
Abandoned mine drainage | Abandoned mine drainage (also known as AMD) is a form of water pollution involving water that has been polluted by contact with mines, typically coal mines. Although it is sometimes called "acid mine drainage", not all abandoned mine drainage is acidic.
Passive treatment system | A passive treatment system is a method for removing metals from acid mine drainage. There are several types of passive treatment systems, each of which may be used on their own or in combination to treat effluents. The type of system selected is dependent upon the chemistry of the acid mine drainage and the flow of the disc

In [4]:
print(response.answer)

Question: What are types of skins?
Context:
[1] «Human skin | The human skin is the outer covering of the body. In humans, it is the largest organ of the integumentary system. The skin has up to seven layers of ectodermal tissue and guards the underlying muscles, bones, ligaments and internal organs. Human skin is similar to that of most other mammals. Though nearly all human skin is covered with hair follicles, it can appear hairless. There are two general types of skin, hairy and glabrous skin (hairless). The adjective cutaneous literally means "of the skin" (from Latin "cutis", skin).»
[2] «Baumann Skin Types | The Baumann


In [41]:
question = "Who played the role of Barry Allen in the 2014 TV series The Flash?"
retriever = dspy.Retrieve(k=2)
raw = retriever(question)


rqa = dspy.Predict("question, context -> answer")
response = rqa(question=question, context=raw.passages)

In [42]:
print(response.answer)

Question: Who played the role of Barry Allen in the 2014 TV series The Flash?
Context:
[1] «The Flash (2014 TV series) | The Flash is an American television series developed by Greg Berlanti, Andrew Kreisberg and Geoff Johns, airing on The CW. It is based on the DC Comics character Barry Allen / Flash, a costumed superhero crime-fighter with the power to move at superhuman speeds. It is a spin-off from "Arrow", existing in the same fictional universe. The series follows Allen, portrayed by Grant Gustin, a crime scene investigator who gains super-human speed, which he uses to fight criminals, including others who have also gained superhuman abilities.»
[2]


In [29]:
class GenerateQuery(dspy.Signature):
    """ Generate a search query input question """
    context = dspy.InputField(desc="may contain relevant fact for the question")
    question: str = dspy.InputField()
    query = dspy.OutputField()

class GenerateResponse(dspy.Signature):
    """ Answer question with short factual response """
    context = dspy.InputField(desc="may contain relevant fact for the question")
    question: str = dspy.InputField()
    response = dspy.OutputField(desc="short factual response; 1-5 words")

class MultiHop(dspy.Module):
    def __init__(self, k_passages=3, m_hops=3):
        super().__init__()
        
        self.k_passages = k_passages
        self.m_hops = m_hops
        self.generate_query = dspy.ChainOfThought(GenerateQuery)
        self.retrieve = dspy.Retrieve(k=self.k_passages)
        self.generate_response = dspy.ChainOfThought(GenerateResponse)
        
    def forward(self, question):
        context = []
        for hop in range(self.m_hops):    
            query_res = self.generate_query(context=context, question=question)
            query  = query_res.query
            passages = self.retrieve(query).passages
            context = deduplicate(context + passages)
            
        response = self.generate_response(question=question, context=passages)
        return dspy.Prediction(context=context, response=response.response)    
            
    

In [30]:
question = "Who played the role of Barry Allen in the 2014 TV series The Flash?"
module = MultiHop()
output = module(question=question)


In [31]:
from rich import print as rprint

print("Context: ")
_ = [rprint(p) for p in output.context]

print("\nResponse: ")
rprint(output.response)

Context: 


The Flash (2014 TV series) | The Flash is an American television series developed by Greg Berlanti, Andrew 
Kreisberg and Geoff Johns, airing on The CW. It is based on the DC Comics character Barry Allen / Flash, a costumed
superhero crime-fighter with the power to move at superhuman speeds. It is a spin-off from "Arrow", existing in the
same fictional universe. The series follows Allen, portrayed by Grant Gustin, a crime scene investigator who gains 
super-human speed, which he uses to fight criminals, including others who have also gained superhuman abilities.

List of The Flash characters | "The Flash" is an American television series developed by writer-producers Greg 
Berlanti, Andrew Kreisberg and Geoff Johns, based on the DC Comics character the Flash. The series premiered in the
United States on October 7, 2014, on The CW television network. It is a spin-off from "Arrow", a show in the same 
universe. The first season follows police forensic investigator Barry Allen (Grant Gustin), who develops 
super-speed after he is struck by lightning. In his attempt to learn about his powers and how to use them for good,
he is assisted by S.T.A.R. Labs' Dr. Caitlin Snow (Danielle Panabaker), Cisco Ramon (Carlos Valdes) and Dr. 
Harrison Wells (Tom Cavanagh). Barry also tries to solve the murder of his mother (Michelle Harrison) by a 
superhuman attacker (Matt Letscher) when he was eleven. The murder investigation unjustly imprisoned his father 
(John Wesley Shipp), leaving detective Joe West (Jesse L. Martin), father of his best friend, Iris (Candice 
Patton), to take in the young Barry. The memory of his mother's murder and his father's framing and later killing 
motivates Barry to put his personal needs aside and use his powers to fight those who would hurt the innocent, 
shaping him into the Flash.

Grant Gustin | Thomas Grant Gustin (born January 14, 1990) is an American actor and singer. He is known for his 
role as Barry Allen / the Flash (based on the DC Comics character of the same name) on the CW series "The Flash" 
and "Arrow", both in the Arrowverse television franchise, and his role as Sebastian Smythe on the Fox series 
"Glee".


Response: 


Grant Gustin